In [ ]:
import sys
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy
import time
from datetime import datetime
from pyproj import Transformer, CRS
from shapely.geometry import Polygon, Point
import h5pyd as h5py
from sliderule import icesat2

In [ ]:
# Configure Logging
icesat2_logger = logging.getLogger("sliderule.icesat2")
icesat2_logger.setLevel(logging.INFO)
# Create Console Output
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
icesat2_logger.addHandler(ch)

## SlideRule Configuration

In [ ]:
# Configure ICESat-2 API
icesat2.init("http://44.227.4.67:9081", False)
# Configure Region of Interest
region = [ {"lon":-105.82971551223244, "lat": 39.81983728534918},
           {"lon":-105.30742121965137, "lat": 39.81983728534918},
           {"lon":-105.30742121965137, "lat": 40.164048017973755},
           {"lon":-105.82971551223244, "lat": 40.164048017973755},
           {"lon":-105.82971551223244, "lat": 39.81983728534918} ]

## Execute ATL06 Algorithm using SlideRule

In [ ]:
# Latch Start Time
perf_start = time.perf_counter()

# Build ATL06 Request
parms = {
    "poly": region,
    "srt": icesat2.SRT_LAND,
    "cnf": icesat2.CNF_SURFACE_HIGH,
    "ats": 10.0,
    "cnt": 10,
    "len": 40.0,
    "res": 20.0,
    "maxi": 1
}

# Request ATL06 Data
rsps = icesat2.atl06p(parms, "atl03-cloud")

# Latch Stop Time
perf_stop = time.perf_counter()

# Build Dataframe of SlideRule Responses
df = pd.DataFrame(rsps)

# Display Statistics
perf_duration = perf_stop - perf_start
print("Completed in {:.3f} seconds of wall-clock time". format(perf_duration))
print("Reference Ground Tracks: {}".format(df["rgt"].unique()))
print("Cycles: {}".format(df["cycle"].unique()))
print("Received {} elevations".format(len(df)))

## Plot Region

In [ ]:
# Calculate Extent
lons = [p["lon"] for p in region]
lats = [p["lat"] for p in region]
lon_margin = (max(lons) - min(lons)) * 0.1
lat_margin = (max(lats) - min(lats)) * 0.1
extent = (min(lons) - lon_margin, max(lons) + lon_margin, min(lats) - lat_margin, max(lats) + lat_margin)

# Create Plot
fig = plt.figure(num=None, figsize=(24, 12))
box_lon = [e["lon"] for e in region]
box_lat = [e["lat"] for e in region]

# Plot SlideRule Ground Tracks
ax1 = plt.subplot(121,projection=cartopy.crs.PlateCarree())
ax1.set_title("SlideRule Zoomed Ground Tracks")
ax1.scatter(df["lon"].values, df["lat"].values, s=2.5, c=df["h_mean"], cmap='winter_r', zorder=3, transform=cartopy.crs.PlateCarree())
ax1.set_extent(extent,crs=cartopy.crs.PlateCarree())
ax1.plot(box_lon, box_lat, linewidth=1.5, color='r', zorder=2, transform=cartopy.crs.Geodetic())

# Plot SlideRule Global View
ax2 = plt.subplot(122,projection=cartopy.crs.PlateCarree())
ax2.set_title("SlideRule Global Reference")
ax2.scatter(df["lon"].values, df["lat"].values, s=2.5, color='r', zorder=3, transform=cartopy.crs.PlateCarree())
ax2.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black')
ax2.add_feature(cartopy.feature.LAKES)
ax2.set_extent((-180,180,-90,90),crs=cartopy.crs.PlateCarree())

# Show Plot
plt.show()

## Plot Data Availability

In [ ]:
# Setup Columns
columns = ["reference_photon_lat", "reference_photon_lon", "segment_ph_cnt"]
spots = ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]

# Query ATL03 Files from NASA CMR System
resources = icesat2.cmr(region, short_name='ATL03')

# Read each file
atl03 = {}
for resource in resources:
    try:
        # Read Resource
        print("Reading {} at {}".format(resource, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
        data = {}
        for spot in spots:
            data[spot] = {}
            for column in columns:
                data[spot][column] = icesat2.h5(spot + "/geolocation/" + column,  resource, "atl03-cloud")
        atl03[resource] = data
    except Exception as e:
        print("Error reading {}: {}".format(resource, e))

# Status
complete = len(atl03)
print("Processed {} of {} at {}".format(complete, len(resources), datetime.now().strftime("%Y-%m-%d %H:%M:%S")))

# Count Number of Elements
numelements = {}
for column in columns:
    numelements[column] = 0
for resource in atl03:
    for spot in spots:
        for column in columns:
            numelements[column] += len(atl03[resource][spot][column])

# Preallocate Dataset
gidx = {} # global index
fulldata = {}
for column in columns:
    print("Allocating {} number of {}".format(numelements[column], column))
    fulldata[column] = np.zeros(numelements[column])
    gidx[column] = 0
    
# Copy Data In
for resource in atl03:
    print("Copying in data for {}".format(resource))
    for spot in spots:
        for column in columns:
            offset = gidx[column]
            for idx,entry in enumerate(atl03[resource][spot][column]):
                fulldata[column][offset + idx] = entry
            gidx[column] += len(atl03[resource][spot][column])

# Assign Back Flattened Data
atl03 =  pd.DataFrame(fulldata)

In [ ]:
# Calculate Extent and Bounding Box
lons = [p["lon"] for p in region]
lats = [p["lat"] for p in region]
lon_margin = (max(lons) - min(lons)) * 1.0
lat_margin = (max(lats) - min(lats)) * 1.0
extent = (min(lons) - lon_margin, max(lons) + lon_margin, min(lats) - lat_margin, max(lats) + lat_margin)
box_lon = [e["lon"] for e in region]
box_lat = [e["lat"] for e in region]

# Plot Ground Tracks
fig = plt.figure(num=None, figsize=(24, 12))
ax1 = plt.subplot(111,projection=cartopy.crs.PlateCarree())
ax1.set_title("SlideRule Zoomed Ground Tracks")
ax1.scatter(atl03["reference_photon_lon"].values, atl03["reference_photon_lat"].values, s=2.5, c=atl03["segment_ph_cnt"], cmap='winter_r', zorder=3, transform=cartopy.crs.PlateCarree())
ax1.set_extent(extent,crs=cartopy.crs.PlateCarree())
ax1.plot(box_lon, box_lat, linewidth=1.5, color='r', zorder=2, transform=cartopy.crs.Geodetic())

# Show Plot
plt.show()

## Manually Subset Data (Runs REALLY Long Time)

In [ ]:
# Allocate an Invalid Column
atl03["invalid"] = np.zeros(numelements["segment_ph_cnt"])

# Create Projection Transformer
transformer = Transformer.from_crs(4326, 3857) # GPS to Web Mercator

# Project Polygon
pregion = []
for point in bounding_box:
    ppoint = transformer.transform(point["lat"], point["lon"])
    pregion.append(ppoint)
    
# Trim Results
polygon = Polygon(pregion)
for lat,lon,h in zip(atl03df_full["reference_photon_lat"],atl03df_full["reference_photon_lon"],atl03df_full["segment_ph_cnt"]):
    c = transformer.transform(lat, lon)
    point = Point(c[0], c[1])
    if not point.within(polygon):
        atl03["invalid"] = True

# Filter Invalid Data
atl03 = atl03[atl03["invalid"] == False]